In [61]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [62]:
wb = load_workbook(filename='Boston Neighborhood Coordinates.xlsx', read_only=True)
ws = wb.active
col_names = [ws["A1"].value, ws["B1"].value, ws["C1"].value, ws["D1"].value]
header = np.array(col_names)

df = pd.DataFrame(ws.values, columns=header)
wb.close()

In [63]:
df = df.drop(['Zip Code'], axis=1)
df = df.drop([0])
df.set_index('Neighborhood', inplace=True)

In [64]:
df

Latitude Longitude
Neighborhood                                 
Chinatown/Leather District  42.3515  -71.0602
West End                    42.3653  -71.0646
South End                   42.3378  -71.0706
Roxbury                     42.3218  -71.0884
Mission Hill                42.3319  -71.0988
Mattapan                    42.2771  -71.0914
South Boston                42.3365  -71.0349
East Boston                 42.3745   -71.032
Charlestown                 42.3795  -71.0646
Jamaica Plain               42.3097  -71.1151
Roslindale                  42.2832   -71.127
West Roxbury                42.2798  -71.1627
Allston                     42.3493    -71.13
Brighton                      42.35   -71.158
Hyde Park                   42.2528    -71.13
North Cambridge             42.3933  -71.1345
Brookline Village           42.3328  -71.1389
Chestnut Hill               42.3278  -71.1627
Downtown                    42.3573  -71.0562
North End                   42.3646  -71.0534
Beacon Hill                 42.3598  -71.0678
Back Bay                    42.3515  -71.0805
Dorchester                  42.2981  -71.0625
Harvard Square              42.3725  -71.1202
East Cambridge              42.3677   -71.081
Fenway/Kenmore              42.3441  -71.0957
Porter Square               42.3894  -71.1194
West Cambridge               42.377  -71.1359
Inman Square                42.3734  -71.0982
Central Square              42.3651  -71.1034
Davis Square                42.3963  -71.1222
Winter Hill                 42.3909  -71.0958
Assembly Square             42.3931  -71.0804
Seaport                     42.3492  -71.0415

In [65]:
import folium # map rendering library

In [66]:
# create map of Boston using latitude and longitude values
map_Boston = folium.Map(location=[42.3261, -71.07], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df.index):
    label = f'{neighborhood}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Boston)
    
    folium.Circle([lat, lng],
                    radius=1000
                   ).add_to(map_Boston)
    
map_Boston

In [67]:
import requests
import json # library to handle JSON files

In [68]:
CLIENT_ID = 'GKVJ5KSFU1RD00P24CV5JT2XSARMWSONFYRMLT2EK2IJFEBW' # your Foursquare ID
CLIENT_SECRET = 'SSJALJ4S5QXDNQBZMNUNFBE1XMVQZETMJWU1JDXE2VWK2PZL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [72]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [73]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name          categories        lat        lng
0                Coppa  Italian Restaurant  42.343462 -71.068567
1               Grotto  Italian Restaurant  42.359915 -71.062807
2           Limoncello  Italian Restaurant  42.363353 -71.053827
3            Giacomo's  Italian Restaurant  42.364683 -71.053579
4  Trattoria Il Panino  Italian Restaurant  42.363836 -71.054500

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 1000
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        r = requests.get(url)
        print(r.status_code)
        results = r.json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
Boston_venues = getNearbyVenues(names=df.index, latitudes=df['Latitude'], longitudes=df['Longitude'], radius=1000)

Chinatown/Leather District
200
West End
200
South End
200
Roxbury
200
Mission Hill
200
Mattapan
200
South Boston
200
East Boston
200
Charlestown
200
Jamaica Plain
200
Roslindale
200
West Roxbury
200
Allston
200
Brighton
200
Hyde Park
200
North Cambridge 
200
Brookline Village
200
Chestnut Hill
200
Downtown
200
North End
200
Beacon Hill
200
Back Bay
200
Dorchester
200
Harvard Square 
200
East Cambridge
200
Fenway/Kenmore
200
Porter Square
200
West Cambridge
200
Inman Square
200
Central Square
200
Davis Square
200
Winter Hill
200
Assembly Square
200
Seaport
200


In [75]:
print(Boston_venues.shape)
    
Boston_venues.nunique()

(2683, 7)


Neighborhood                34
Neighborhood Latitude       34
Neighborhood Longitude      31
Venue                     1908
Venue Latitude            2169
Venue Longitude           2169
Venue Category             297
dtype: int64

In [76]:
Boston_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Chinatown/Leather District                42.3515                -71.0602   
1  Chinatown/Leather District                42.3515                -71.0602   
2  Chinatown/Leather District                42.3515                -71.0602   
3  Chinatown/Leather District                42.3515                -71.0602   
4  Chinatown/Leather District                42.3515                -71.0602   

                             Venue  Venue Latitude  Venue Longitude  \
0                 Gracenote Coffee       42.351657       -71.058204   
1                        Shabu Zen       42.350922       -71.060942   
2                 Barry's Bootcamp       42.354010       -71.059776   
3  Hostelling International Boston       42.351084       -71.063398   
4              Tatte Bakery & Cafe       42.353111       -71.057117   

         Venue Category  
0           Coffee Shop  
1      Asian Restaurant  
2  Gym / Fitness Center  
3                Hostel  
4                Bakery

In [77]:
Venue_cat_count = Boston_venues.groupby('Venue Category').count()
Venue_cat_count.rename(columns={'Venue': 'Count'}, inplace=True)
Venue_cat_count = pd.DataFrame(Venue_cat_count['Count'], Venue_cat_count.index)
Venue_cat_count.to_excel('Venue_Category_Counts.xlsx')

In [78]:
Venue_per_hood = Boston_venues.groupby('Neighborhood').count()
Venue_per_hood.rename(columns={'Venue': 'Count'}, inplace=True)
Venue_per_hood = pd.DataFrame(Venue_per_hood['Count'], Venue_per_hood.index)
Venue_per_hood.to_excel('Venue_Per_Hood.xlsx')

In [79]:
# one hot encoding
Boston_onehot = pd.get_dummies(Boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Boston_onehot['Neighborhood'] = Boston_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Boston_onehot.columns[-1]] + list(Boston_onehot.columns[:-1])
Boston_onehot = Boston_onehot[fixed_columns]

Totals = Boston_onehot.sum()
Totals = pd.DataFrame(Totals)
Totals = Totals.drop('Neighborhood')
Totals.rename(columns={0: 'Totals'}, inplace=True)

In [80]:
Boston_grouped = Boston_onehot.groupby('Neighborhood').mean().reset_index()
Boston_grouped

Neighborhood  Zoo Exhibit      ATM  Accessories Store  \
0                      Allston     0.000000  0.00000           0.000000   
1              Assembly Square     0.000000  0.00000           0.000000   
2                     Back Bay     0.000000  0.00000           0.000000   
3                  Beacon Hill     0.000000  0.00000           0.000000   
4                     Brighton     0.000000  0.00000           0.000000   
5            Brookline Village     0.000000  0.00000           0.000000   
6               Central Square     0.000000  0.00000           0.000000   
7                  Charlestown     0.000000  0.00000           0.000000   
8                Chestnut Hill     0.000000  0.00000           0.000000   
9   Chinatown/Leather District     0.000000  0.00000           0.000000   
10                Davis Square     0.000000  0.00000           0.000000   
11                  Dorchester     0.000000  0.00000           0.000000   
12                    Downtown     0.000000  0.00000           0.000000   
13                 East Boston     0.000000  0.00000           0.000000   
14              East Cambridge     0.012821  0.00000           0.000000   
15              Fenway/Kenmore     0.000000  0.00000           0.000000   
16             Harvard Square      0.000000  0.00000           0.000000   
17                   Hyde Park     0.000000  0.02381           0.000000   
18                Inman Square     0.000000  0.00000           0.000000   
19               Jamaica Plain     0.000000  0.00000           0.024691   
20                    Mattapan     0.000000  0.00000           0.000000   
21                Mission Hill     0.000000  0.00000           0.000000   
22            North Cambridge      0.000000  0.00000           0.000000   
23                   North End     0.000000  0.00000           0.000000   
24               Porter Square     0.000000  0.00000           0.000000   
25                  Roslindale     0.000000  0.00000           0.000000   
26                     Roxbury     0.000000  0.00000           0.000000   
27                     Seaport     0.000000  0.00000           0.000000   
28                South Boston     0.000000  0.00000           0.000000   
29                   South End     0.000000  0.00000           0.010000   
30              West Cambridge     0.000000  0.00000           0.000000   
31                    West End     0.000000  0.00000           0.000000   
32                West Roxbury     0.000000  0.00000           0.000000   
33                 Winter Hill     0.000000  0.00000           0.000000   

    Adult Boutique  Afghan Restaurant  African Restaurant   Airport  \
0          0.00000           0.000000            0.000000  0.000000   
1          0.00000           0.000000            0.000000  0.000000   
2          0.00000           0.000000            0.000000  0.000000   
3          0.00000           0.000000            0.000000  0.000000   
4          0.00000           0.000000            0.000000  0.000000   
5          0.00000           0.000000            0.000000  0.000000   
6          0.00000           0.000000            0.000000  0.000000   
7          0.00000           0.000000            0.000000  0.000000   
8          0.00000           0.000000            0.000000  0.000000   
9          0.00000           0.000000            0.000000  0.000000   
10         0.00000           0.000000            0.000000  0.000000   
11         0.00000           0.000000            0.000000  0.000000   
12         0.00000           0.000000            0.000000  0.000000   
13         0.00000           0.000000            0.000000  0.013699   
14         0.00000           0.012821            0.000000  0.000000   
15         0.00000           0.000000            0.000000  0.000000   
16         0.00000           0.000000            0.000000  0.000000   
17         0.00000           0.000000            0.000000  0.000000   
18         0.00000           0.000000      

In [81]:
num_top_venues = 20

for hood in Boston_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Boston_grouped[Boston_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allston----
                   venue  freq
0        Thai Restaurant  0.04
1                 Bakery  0.03
2       Sushi Restaurant  0.03
3            Coffee Shop  0.03
4    Fried Chicken Joint  0.03
5                    Bar  0.03
6      Korean Restaurant  0.03
7     Mexican Restaurant  0.03
8            Pizza Place  0.03
9     Chinese Restaurant  0.03
10   Japanese Restaurant  0.02
11          Liquor Store  0.02
12             Gift Shop  0.02
13         Grocery Store  0.02
14                   Gym  0.02
15  Gym / Fitness Center  0.02
16        Ice Cream Shop  0.02
17    Falafel Restaurant  0.02
18                  Café  0.02
19            Donut Shop  0.02


----Assembly Square----
                     venue  freq
0               Donut Shop  0.06
1           Clothing Store  0.06
2       Mexican Restaurant  0.05
3              Coffee Shop  0.04
4      American Restaurant  0.03
5              Pizza Place  0.03
6                     Café  0.03
7      Sporting Goods Shop  0.03
8         

In [82]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [83]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Boston_grouped['Neighborhood']

for ind in np.arange(Boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Boston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

Neighborhood    1st Most Common Venue  \
0                      Allston          Thai Restaurant   
1              Assembly Square               Donut Shop   
2                     Back Bay           Clothing Store   
3                  Beacon Hill       Italian Restaurant   
4                     Brighton              Pizza Place   
5            Brookline Village              Coffee Shop   
6               Central Square           Sandwich Place   
7                  Charlestown                     Café   
8                Chestnut Hill       Italian Restaurant   
9   Chinatown/Leather District              Coffee Shop   
10                Davis Square       Mexican Restaurant   
11                  Dorchester    Vietnamese Restaurant   
12                    Downtown                     Park   
13                 East Boston       Italian Restaurant   
14              East Cambridge           Science Museum   
15              Fenway/Kenmore      American Restaurant   
16             Harvard Square                      Café   
17                   Hyde Park      American Restaurant   
18                Inman Square  New American Restaurant   
19               Jamaica Plain                     Park   
20                    Mattapan     Caribbean Restaurant   
21                Mission Hill               Donut Shop   
22            North Cambridge                      Café   
23                   North End       Italian Restaurant   
24               Porter Square      Japanese Restaurant   
25                  Roslindale              Pizza Place   
26                     Roxbury      American Restaurant   
27                     Seaport       Seafood Restaurant   
28                South Boston                     Park   
29                   South End       Italian Restaurant   
30              West Cambridge      American Restaurant   
31                    West End       Italian Restaurant   
32                West Roxbury       Italian Restaurant   
33                 Winter Hill              Pizza Place   

     2nd Most Common Venue    3rd Most Common Venue  4th Most Common Venue  \
0         Sushi Restaurant              Coffee Shop            Pizza Place   
1           Clothing Store       Mexican Restaurant            Coffee Shop   
2                      Spa       Italian Restaurant                   Park   
3                     Park              Coffee Shop          Historic Site   
4       Chinese Restaurant               Donut Shop            Coffee Shop   
5       Mexican Restaurant                     Park     Chinese Restaurant   
6       Italian Restaurant  New American Restaurant                 Bakery   
7                Gastropub                     Park            Pizza Place   
8   Furniture / Home Store       Seafood Restaurant         Baseball Field   
9                   Bakery       Chinese Restaurant         Sandwich Place   
10          Breakfast Spot      Japanese Restaurant                   Café   
11                    Café                     Park             Restaurant   
12      Italian Restaurant                    Hotel            Coffee Shop   
13      Mexican Restaurant              Pizza Place         Airport Lounge   
14     American Restaurant              Pizza Place                   Park   
15                    Café                    Hotel           Burger Joint   
16     American Restaurant                Bookstore                   Park   
17             Pizza Place             Skating Rink         Baseball Field   
18             Coffee Shop                      Pub  Portuguese Restaurant   
19             Coffee Shop                   Bakery    American Restaurant   
20              Donut Shop                 Pharmacy            Pizza Place   
21          Sandwich Place                     Café             Art Museum   
22           Grocery Store       Mexican Restaurant            Yoga Studio   
23      Seafood Restaurant                     Park            Pizza Place   
24

In [84]:
def df_style(val):
    boolean = 0
    for item in match:
        if val == item:
            boolean = 1
        
    if boolean == 1:
        return 'font-weight: bold'
    else:
        return ''

### Create df that includes Neighborhoods with MAIN criterion

In [85]:
criterion = ['Italian', 'Pizza', 'Ice Cream', 'Gym', 'Pub']
match = []
neighborhoods_venues_sorted.sort_values(by=['Neighborhood'], inplace=True)


mask = neighborhoods_venues_sorted.applymap(lambda x: criterion[0] in str(x))
search_results = neighborhoods_venues_sorted[mask == True].dropna(how='all')

for i in range(0,len(search_results.index)):
    results = []
    results = search_results.iloc[i,].unique()
    match.insert(0, results[1])
match = list(dict.fromkeys(match))

search_results = neighborhoods_venues_sorted.iloc[search_results.index]
    
i=0
for items in zip(search_results['Neighborhood']):
    i=i+1
print(f"{i} Results Found!")

search_results.style.applymap(df_style)

21 Results Found!


### Score Neighborhoods based on MAIN criterion

In [86]:
main_score = []
for row in search_results.itertuples():
    i=0
    for item in zip(row):
        if item[0] == match[0]:
            if i <= 2:
                main_score.append(1)
            elif i <= 4:
                main_score.append(.9)
            elif i <= 6:
                main_score.append(.75)
            elif i <= 11:
                main_score.append(.5)
            elif i <= 16:
                main_score.append(.33)
            elif i <= 21:
                main_score.append(.2)
            else:
                print("ERROR: i larger than expected")
        i=i+1
                
search_results['Main Score'] = main_score
search_results.sort_values(by=['Main Score'], ascending=False, inplace=True)
search_results.style.applymap(df_style)

C:\Users\jhenn\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jhenn\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Create match list seperately

In [87]:
match = []

for item in criterion:
    
    mask = neighborhoods_venues_sorted.applymap(lambda x: item in str(x))
    match_results = neighborhoods_venues_sorted[mask == True].dropna(how='all')

    for i in range(0,len(match_results.index)):
        results = []
        results = match_results.iloc[i,].unique()
        match.insert(0, results[1])
    match = list(dict.fromkeys(match))

### Get df with all neighborhoods organized by # of criterion met with criterion bolded

In [88]:
num_match = []
match_score = []
search_results_all_hood = neighborhoods_venues_sorted

for row in search_results_all_hood.itertuples():
    i=0
    for item in zip(row):
        for crit in match:
            if item[0] == crit:
                i=i+1
    num_match.append(i)
    match_score.append(i/5)
search_results_all_hood['Number of Matches'] = num_match
search_results_all_hood['Match Score'] = match_score
search_results_all_hood.sort_values(by=['Number of Matches'], ascending=False, inplace=True)

search_results_all_hood.style.applymap(df_style)

### Create df that includes Neighborhoods with ALL criterion

In [89]:
search_results_perf = search_results_all_hood.loc[search_results_all_hood['Number of Matches'] == 5]

rows = len(search_results_perf.index)

print(f"{rows} Results Found!")
if rows == 0:
    print("Criterion too specific")
else:
    for i in range(0,rows):
        main_score1 = search_results.loc[search_results_perf.index[i], 'Main Score']
        match_score1 = search_results_all_hood.loc[search_results_perf.index[i], 'Match Score']
    tot_score = (main_score1 + match_score1)/2
    search_results_perf['Main Score'] = main_score1
    search_results_perf['Match Score'] = main_score1
    search_results_perf['Total Score'] = tot_score
    search_results_perf.sort_values(by=['Total Score'], ascending=False, inplace=True)
    search_results_perf.style.applymap(df_style)

1 Results Found!


C:\Users\jhenn\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\jhenn\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jhenn\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

# Map all Italian Restaurants in Boston

In [112]:
Boston_Italian = Boston_venues.loc[Boston_venues['Venue Category'] == 'Italian Restaurant']
Boston_Italian.drop(['Neighborhood Latitude', 'Neighborhood Longitude'], axis=1, inplace=True)
fixed_columns = [Boston_Italian.columns[1]] + list(Boston_Italian.columns[2:4]) + [Boston_Italian.columns[0]] + [Boston_Italian.columns[-1]]
Boston_Italian = Boston_Italian[fixed_columns]

C:\Users\jhenn\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [117]:
Boston_Italian

Venue  Venue Latitude  Venue Longitude  \
80        Maggiano's Little Italy       42.350966       -71.068695   
112               Osteria Rustico       42.364397       -71.059870   
121   Scampo at The Liberty Hotel       42.361937       -71.070513   
122                        Grotto       42.359915       -71.062807   
176                          Aria       42.363286       -71.055002   
...                           ...             ...              ...   
2618                    Sportello       42.350519       -71.048687   
2650            Sapóre Ristorante       42.348126       -71.039863   
2652               Buco Trattoria       42.349923       -71.049393   
2656               Tuscan Kitchen       42.352829       -71.047298   
2670            Strega Waterfront       42.353062       -71.045162   

                    Neighborhood      Venue Category  
80    Chinatown/Leather District  Italian Restaurant  
112                     West End  Italian Restaurant  
121                     West End  Italian Restaurant  
122                     West End  Italian Restaurant  
176                     West End  Italian Restaurant  
...                          ...                 ...  
2618                     Seaport  Italian Restaurant  
2650                     Seaport  Italian Restaurant  
2652                     Seaport  Italian Restaurant  
2656                     Seaport  Italian Restaurant  
2670                     Seaport  Italian Restaurant  

[89 rows x 5 columns]

In [120]:
# create map of Boston using latitude and longitude values
map_Italian = folium.Map(location=[42.3261, -71.07], zoom_start=12)

# add markers to map
for lat, lng, venue, neighborhood in zip(Boston_Italian['Venue Latitude'], Boston_Italian['Venue Longitude'], Boston_Italian['Venue'], Boston_Italian['Neighborhood']):
    label = f'{venue}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Italian)
    
map_Italian